# Assignment 3 - Extra : CIFAR-10

https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

## Import Data

In [ ]:
! pip install tensorflow==2.3

     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 6.8MB 45.6MB/s 
     |████████████████████████████████| 460kB 52.1MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
! nvidia-smi

Sat Aug  1 09:56:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=78f0fa12b26c198d67d5e7916ab768d13c8cd72bdf977440b7e0144a3e042a82
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [ ]:
import numpy as np
import pickle
import tensorflow as tf

time: 1.21 s


In [ ]:
tf.__version__

'2.3.0'

time: 6.96 ms


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step
time: 5.64 s


In [ ]:
X_train.shape

(50000, 32, 32, 3)

time: 7.72 ms


## Preprocessing

In [ ]:
import tensorflow as tf

ImgDatagenProperty = {
    # 'rescale' : 1/255,
    # 'rotation_range' : 40,
    # 'width_shift_range' : 0.4,
    # 'height_shift_range' : 0.4,
    'shear_range' : 0.2,
    # 'zoom_range' : 0.3,
    # 'horizontal_flip' : True,
    # 'fill_mode' : 'nearest',
    'zca_whitening' : True,
    'preprocessing_function' : tf.keras.applications.efficientnet.preprocess_input,
}

time: 1.85 ms


In [ ]:
ImgValDatagenProperty = {
    # 'rescale' : 1/255,
    'preprocessing_function' : tf.keras.applications.efficientnet.preprocess_input,
}

time: 1.56 ms


In [ ]:
ImgModelProperty = {
    'img_height' : 32,
    'img_width' : 32,
    'num_class' : 10,
    'epoch' : 60,
    'batch_size' : 128
}

time: 2.96 ms


In [ ]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            zca_epsilon=1e-06,
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.2,
            fill_mode='nearest',
            horizontal_flip=True,**ImgDatagenProperty).flow(X_train, y_train, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**ImgValDatagenProperty).flow(X_test, y_test, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


time: 840 ms


## Model

https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet101

In [ ]:
import tensorflow.keras.backend as K

K.clear_session()

time: 6.48 ms


In [ ]:
import tensorflow as tf

def BuildModel(input_dim = (32, 32, 3), n_class = 10) -> tf.keras.Sequential :
    model = tf.keras.Sequential()
    model.add(tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_shape=input_dim))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    # model.add(tf.keras.layers.Flatten())
    # model.add(tf.keras.layers.Dense(128, activation=tf.keras.activations.relu))
    # model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(64, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.BatchNormalization())
    # model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))
    # model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax))
    # padding_input = tf.keras.layers.ZeroPadding2D(padding=(96, 96), input_shape=input_dim)
    # pretrained = tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')(padding_input)

    # model = tf.keras.Model(padding_input.input, pretrained)
    return model    

time: 9.41 ms


In [ ]:
model = BuildModel()

27025408/27018416 [==============================] - 0s 0us/step
time: 4.71 s


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 1, 1, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 6,658,129
Trainable params: 6,595,946
Non-trainable params: 62,183
_________________________________________________________________
time: 22.5 ms


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

time: 25.1 ms


In [ ]:
import tensorflow as tf

red = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='auto', min_lr=1e-7)

time: 1.62 ms


In [ ]:
%load_ext tensorboard
! rm -rf ./logs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
time: 2.02 s


In [ ]:
import tensorflow as tf

esb = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

time: 3.03 ms


In [ ]:
import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tsb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

time: 4.51 ms


In [ ]:
model.fit(
    train_datagen, 
    epochs=ImgModelProperty['epoch'],
    steps_per_epoch=len(X_train) / ImgModelProperty['batch_size'],
    validation_data=val_datagen,
    callbacks=[red, tsb, esb],
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/60
391/390 [==============================] - 42s 107ms/step - loss: 1.1929 - sparse_categorical_accuracy: 0.5920 - val_loss: 0.8732 - val_sparse_categorical_accuracy: 0.7284
Epoch 2/60
391/390 [==============================] - 40s 101ms/step - loss: 0.7167 - sparse_categorical_accuracy: 0.7517 - val_loss: 0.6713 - val_sparse_categorical_accuracy: 0.7862
Epoch 3/60
391/390 [==============================] - 39s 100ms/step - loss: 0.6046 - sparse_categorical_accuracy: 0.7921 - val_loss: 0.5409 - val_sparse_categorical_accuracy: 0.8183
Epoch 4/60
391/390 [==============================] - 40s 103ms/step - loss: 0.5418 - sparse_categorical_accuracy: 0.8120 - val_loss: 0.6212 - val_sparse_categorical_accuracy: 0.8048
Epoch 5/60
391/390 [==============================] - 41s 105ms/step - loss: 0.4966 - sparse_categorical_accuracy: 0.8282 - val_loss: 0.4971 - val_sparse_categorical_accuracy: 0.8339
Epoch 6/60
391/390 [==============================] - 40s 102ms/step - loss: 0.4691 -

time: 15min 24s


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
y_pred = model.predict(val_datagen)

time: 2.8 s


In [ ]:
model.evaluate(val_datagen)

79/79 [==============================] - 2s 20ms/step - loss: 0.3848 - sparse_categorical_accuracy: 0.8875


[0.38483357429504395, 0.887499988079071]

time: 1.7 s
